# Flow modelling – Altering flows

In [477]:
import graph_tool as gt
import graph_tool.draw
import graph_tool.search
import numpy as np
import datetime
import cairo

from collections import deque

In [478]:
print("Date:", datetime.datetime.now().date().isoformat())

Date: 2018-11-27


## Test data

Let's create an test graph that models cucumber production, packaging and consumption.

In [479]:
G = gt.Graph(directed=True)
G.load("cucumba.gt")

## Combining solutions or solution-parts

A **solution** is defined by
1. amount of change
2. affected material
3. the edge on which the solution is applied

**Solution-graph**: A subset of the graph/system containing only the edges that are affected by the solution and on which a solution has been already applied, thus the flows have been altered.

**Solution-part**: A solution can have several parts in case the desired change in a material results in changes in other materials. For example reducing the milk production also reduces the plastic production.

Once we computed the solution-parts for the milk and for the plastic, we need to integrate both into a single graph which then represents *the* complete solution. In a similar fashion, it should be possible to combine solutions that are part of a strategy.

The straightforward approach is to sum up the mass changes of each solution graph. In which case:

    for each edge:

        final mass = original mass + solution 1 + ... + solution n
        
But this requires that solution_n is the signed amount of change in mass for a particular edge, not the amount after applying the solution.

In [224]:
for i,e in enumerate(g_split.edges()):
    formula = "{eid}    {omass} + {s1}[mask {m1}] + {s2}[mask {m2}] + {s3}[mask {m3}] = {res}".format(
        eid=i, omass=eprop_mass[e],
        s1=round(mass_sol1[e],2), s2=round(mass_sol2[e],2), 
        s3=round(mass_sol3[e],2), m1=edge_mask_1[e], m2=edge_mask_2[e], m3=edge_mask_3[e],
        res=round(sum([eprop_mass[e], mass_sol1[e], mass_sol2[e], mass_sol3[e]]), 2)
    )
    print(formula)

0    30.001 + 0.0[mask 0] + 0.0[mask 0] + 0.0[mask 0] = 30.0
1    64.99900000000001 + 0.0[mask 0] + -26.0[mask 1] + 0.0[mask 0] = 39.0
2    5.0 + -0.3[mask 1] + 0.0[mask 0] + -0.6[mask 1] = 4.1
3    30.0 + 0.0[mask 0] + 0.0[mask 0] + 0.0[mask 0] = 30.0
4    65.0 + 0.0[mask 0] + -26.0[mask 1] + 0.0[mask 0] = 39.0
5    20.0 + -1.2[mask 1] + 0.0[mask 0] + -0.6[mask 1] = 18.2
6    16.0 + 0.0[mask 0] + 0.0[mask 0] + 0.0[mask 0] = 16.0
7    4.0 + -0.24[mask 1] + 0.0[mask 0] + -2.41[mask 1] = 1.35
8    5.0 + -0.3[mask 1] + 0.0[mask 0] + -0.6[mask 1] = 4.1
9    75.0 + 0.0[mask 0] + -30.0[mask 1] + 0.0[mask 0] = 45.0
10    3.0 + -0.18[mask 1] + 0.0[mask 0] + -0.36[mask 1] = 2.46
11    2.0 + -0.12[mask 1] + 0.0[mask 0] + -0.24[mask 1] = 1.64
12    20.0 + 0.0[mask 0] + -8.0[mask 1] + 0.0[mask 0] = 12.0
13    1.0 + 0.0[mask 0] + 0.0[mask 0] + 0.0[mask 0] = 1.0
14    1.0 + -0.06[mask 1] + 0.0[mask 0] + -0.24[mask 1] = 0.7


In [225]:
for i,e in enumerate(g_split.edges()):
    formula = "{eid}    {src} --> {dst}    change:  {ch}t  {mat}".format(
        eid=i, src=g_split.vp.id[e.source()], dst=g_split.vp.id[e.target()], mat=eprop_material[e], 
        ch=round(sum([mass_sol1[e],mass_sol2[e],mass_sol3[e]]), 2)
    )
    print("")
    print(formula)


0    Farm --> Packaging    change:  0.0t  cucumber

1    Farm --> Packaging    change:  -26.0t  milk

2    Packaging --> Consumption    change:  -0.9t  plastic

3    Packaging --> Consumption    change:  0.0t  cucumber

4    Packaging --> Consumption    change:  -26.0t  milk

5    Oil rig --> Oil refinery    change:  -1.8t  crude oil

6    Oil refinery --> Stock 1    change:  0.0t  petrol

7    Oil refinery --> Production    change:  -2.65t  plastic

8    Production --> Packaging    change:  -0.9t  plastic

9    Consumption --> Waste    change:  -30.0t  human waste

10    Consumption --> Burn    change:  -0.54t  plastic

11    Consumption --> Recycling    change:  -0.36t  plastic

12    Consumption --> Waste 2    change:  -8.0t  other waste

13    Recycling --> Stock 2    change:  0.0t  waste

14    Recycling --> Production    change:  -0.3t  plastic
